<a href="https://colab.research.google.com/github/yandod/machine-learning-examples/blob/main/08_openai_completion_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* このノートはColabのシークレット内にOpenAIのAPIキーが設定されていることを前提としています
* OPENAI_KEY というシークレットを作成し、[有効なAPIキー](https://platform.openai.com/settings/profile?tab=api-keys)を設定してください





In [1]:
#@title 1.スプレッドシートからデータを読み込む
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet_key = '1qTZdjLmSrL8C2AFL6ap9bTMJrWo6iO3D_tv8AQAGqsg' #@param {type:"string"}

#最初のシートを指定
worksheet = gc.open_by_key(worksheet_key).sheet1

data = worksheet.get_all_values()
headers = data[0]  # 一行目をヘッダーとして取得
rows = data[1:]  # 二行目以降をデータとして取得

# データフレームに変換し、一行目をインデックスとして設定
import pandas as pd
df = pd.DataFrame(rows, columns=headers).set_index(headers[0])
df

,質問,模範解答,理由
id,,,
1,ネコが出てくる短い話を140文字以内で作ってください,むかしむかし、あるところにとてもおりこうなネコがいました。おりこうなネコはみんなを幸せにしま...,ネコが主要な登場人物かどうか、全体で140文字に収まっているかどうか
2,次の言葉をひらがなに直してください\n東京特許許可局,とうきょうとっきょきょかきょく,正確にひらがなに直っているから
3,清泉女子大学はどこにある,清泉女子大学は、東京都品川区東五反田三丁目16番21号に本部を置く日本の私立大学,品川区東五反田三丁目16番21号が正しい所在地


In [2]:
#@title 2.OpenAI ライブラリのインストール
!pip install openai

from openai import OpenAI
from google.colab import userdata

# OpenAI APIキーを設定
client = OpenAI(api_key=userdata.get('OPENAI_KEY'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [3]:
#@title 3.OpenAIのAPIで返答を得る

#モデルの選択
model_name = "gpt-3.5-turbo" #@param ['gpt-3.5-turbo','gpt-4-turbo','gpt-4o']

def get_openai_response(prompt):
  completion = client.chat.completions.create(
  model=model_name,
  messages=[
      {
          "role": "user",
          "content": prompt,
      },
  ],
  )
  return completion.choices[0].message.content

# データフレームに新しい列を追加して、各質問に対するOpenAIの出力を保存
df['response'] = df['質問'].apply(get_openai_response)

# データフレームの表示
df

,質問,模範解答,理由,response
id,,,,
1,ネコが出てくる短い話を140文字以内で作ってください,むかしむかし、あるところにとてもおりこうなネコがいました。おりこうなネコはみんなを幸せにしま...,ネコが主要な登場人物かどうか、全体で140文字に収まっているかどうか,夜、窓辺でネコを見かけた。その目は見つめる私を通り抜け、深い闇の中へ消えた。不思議な感覚に魅...
2,次の言葉をひらがなに直してください\n東京特許許可局,とうきょうとっきょきょかきょく,正確にひらがなに直っているから,とうきょうとっきょきょかきょく
3,清泉女子大学はどこにある,清泉女子大学は、東京都品川区東五反田三丁目16番21号に本部を置く日本の私立大学,品川区東五反田三丁目16番21号が正しい所在地,清泉女子大学は、日本の埼玉県所沢市に位置しています。


In [4]:
#@title 4.OpenAIのAPIで返答と模範解答を比べて採点する

#モデルの選択
model_name = "gpt-4o" #@param ['gpt-3.5-turbo','gpt-4-turbo','gpt-4o']

def get_openai_grade(row):

  prompt = f"""
  下記の出力が質問に沿っているかを模範解答も参考に１から5で採点してください。
  評価には理由も考慮してください。
  質問: {row['質問']}
  出力: {row['response']}
  模範解答: {row['模範解答']}
  評価基準: {row['理由']}
  """
  completion = client.chat.completions.create(
  model=model_name,
  messages=[
      {
          "role": "user",
          "content": prompt,
      },
  ],
  )
  return completion.choices[0].message.content

# データフレームに新しい列を追加して、各質問に対するOpenAIの出力を保存
df['grade'] = df.apply(get_openai_grade, axis=1)

# データフレームの表示
df

,質問,模範解答,理由,response,grade
id,,,,,
1,ネコが出てくる短い話を140文字以内で作ってください,むかしむかし、あるところにとてもおりこうなネコがいました。おりこうなネコはみんなを幸せにしま...,ネコが主要な登場人物かどうか、全体で140文字に収まっているかどうか,夜、窓辺でネコを見かけた。その目は見つめる私を通り抜け、深い闇の中へ消えた。不思議な感覚に魅...,### 評価: 4\n\n#### 理由:\n1. **ネコが主要な登場人物かどうか**: ...
2,次の言葉をひらがなに直してください\n東京特許許可局,とうきょうとっきょきょかきょく,正確にひらがなに直っているから,とうきょうとっきょきょかきょく,評価: 5\n\n理由: 出力は与えられた言葉を正確にひらがなに直しており、模範解答と一致し...
3,清泉女子大学はどこにある,清泉女子大学は、東京都品川区東五反田三丁目16番21号に本部を置く日本の私立大学,品川区東五反田三丁目16番21号が正しい所在地,清泉女子大学は、日本の埼玉県所沢市に位置しています。,評価：1\n\n理由：\n出力された情報は誤りです。清泉女子大学は東京都品川区東五反田三丁目...


In [ ]:
#@title 5.処理結果をCSVで保存してダウンロード
from google.colab import files
import csv
df.to_csv('data.csv', quoting=csv.QUOTE_ALL)
files.download('data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>